# Log
- Written by Junhyeon Kang / email : junhyeon@tesser.co.kr
- Written date: 20230622

----------

# Code Description 
- Use arithmetic voting to draw ground truth segmentations for LNDb 

---------------

# Code Flow
Repeat the following: 
- Select segmentation masks for a single scan
- Load those masks as arrays
- Use voting to draw a single new mask
- Save new masks as nifti files



-------

# Package Import

In [ ]:
import SimpleITK as sitk
import numpy as np
import os, shutil
import nibabel as nib
from collections import defaultdict

-----------

# Utils Functions

In [ ]:
def load_itk(filename):
    print('start load')
    itkimage = sitk.ReadImage(filename)
    
    print('get array now')
    # Convert the image to a  numpy array (dimensions are shuffled as z,y,x)
    itk_array = sitk.GetArrayFromImage(itkimage)

    return itkimage, itk_array

In [ ]:
def write_itk(data_array, input_scan, output_filepath):
        
    result_image = sitk.GetImageFromArray(data_array)
    result_image.CopyInformation(input_scan)

    # write the image
    sitk.WriteImage(result_image, output_filepath+'.nii.gz')

In [ ]:
def neighbor(array, index):
    ind1, ind2, ind3 = index
    print(ind1, ind2, ind3)
    # there will be no boundary segmentations (suppose)
    relevant_box = array[ind1-10 : ind1+11, ind2-10 : ind2+11, ind3-10 : ind3+11]
    # print('rel box is ')
    # print(relevant_box)
    
    maxvalue = np.max(relevant_box)
    print('max value is: ', maxvalue)

    if maxvalue > 0:
        return True, maxvalue
    return False, None

----------

In [ ]:
dir = '/mnt/tesser_nas2/AI_DataSets/lg_tm_CT_LNDb/masks/'
output_dir = '/mnt/tesser_nas2/AI_DataSets/lg_tm_CT_LNDb/inter_masks/'

filelist = os.listdir(dir)
filelist.sort()
filelist

In [ ]:
def run(scan_list):
    image1, array1 = load_itk(dir+scan_list[0])
    array2 = load_itk(dir+scan_list[1])[1]
    
    try:
        array3 = load_itk(dir+scan_list[2])[1]
        inter_array = np.where(array1*array2 + array2*array3 + array3*array1 != 0,\
        -1, 0)
    except:
        inter_array = np.where(array1*array2 != 0,\
        -1, 0)

    
    inter_indices = np.argwhere(inter_array != 0)

    value = 0

    for inter_index in inter_indices:
        ind1, ind2, ind3 = inter_index
        print('indices are: ', [ind1,ind2,ind3])
        print('pixel value is: ', inter_array[ind1,ind2,ind3])
        
        if neighbor(inter_array, inter_index)[0]:
            inter_array[ind1,ind2,ind3] = neighbor(inter_array, inter_index)[1]
                
                
        else: # alone
            value += 1
            print('value is : ', value)

            inter_array[ind1,ind2,ind3] = value
    
    write_itk(inter_array, image1, output_dir+scan_list[0][:-9]+'_inter')


In [ ]:
output_list = os.listdir(output_dir)
scans_for_inter = []

for file in filelist[1:]:
    if file[:9]+'_inter.nii.gz' not in output_list:
        print(file)
        print('scanslist is ', scans_for_inter)
        # add to scans list if not new scan number
        if len(scans_for_inter) > 0:
            if file[:9] in ''.join(scans_for_inter): 
                if file[-1] == 'd': # another radiologist!
                    scans_for_inter.append(file)
                if file == filelist[-1]:
                    run(scans_for_inter)
                    
            else: # new scan
                if len(scans_for_inter) == 1: # then just copy as intersection image too
                    input_image, input_array = load_itk(dir+scans_for_inter[0])
                    write_itk(input_array, input_image, dir+scans_for_inter[0][:-9]+'_inter')
                    
                else: # make an intersection image
                    run(scans_for_inter)
                scans_for_inter = [file]
        
        
        elif len(scans_for_inter) == 0:
            scans_for_inter.append(file)
    
        